In [40]:
import numpy as np
import csv
import pandas as pd
import os
import matplotlib.pyplot as plt
from polyfuzz import PolyFuzz
import random
import json

import productmatching
PATH = os.getcwd()

In [100]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
base_folder = 'aws/dev-dashy-refined-datasets/'
df_all_merged = productmatching.read_file_from_csv(base_folder + "analytics/products", "all-merged.csv")
#df_matches = productmatching.read_file_from_csv(base_folder + "matching", "matches.csv")
df_matches = pd.read_csv('/Users/hassansaeed/Documents/DashyDash/Pandora/matches.csv', parse_dates=True, infer_datetime_format=True, na_filter=True, quoting=csv.QUOTE_ALL, thousands=',')
df_matches = productmatching.getNormalisedProductColumnDType(df_matches)

/var/folders/rj/rrq458bj1mb9fpw62rmd9yb80000gn/T/ipykernel_52721/683553213.py:2: DtypeWarning: Columns (17,18,19,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df_all_merged = productmatching.read_file_from_csv(base_folder + "analytics/products", "all-merged.csv")


read 68136  rows from aws/dev-dashy-refined-datasets/analytics/products all-merged.csv


In [106]:
ranges = [(95,99), (90,95), (85,90), (80,85), (75,80)]
def foo(x):
    if x >= 75 and x < 80:
        return '75-80'
    if x >=80 and x < 85:
        return '80-85'
    if x >=85 and x < 90:
        return '85-90'
    if x >=90 and x < 95:
        return '90-95'
    if x >=95 and x <= 100:
        return '95-99'
    else:
        return '75% or less'
_1 = df_matches[(~df_matches['notes'].isin(['UPC', 'SKU', 'GTIN', 'MPN']))]
_2 = df_matches[(df_matches['notes'].isin(['UPC', 'SKU', 'GTIN', 'MPN']))]
_1['confidence_intervals'] = _1['confidence'].apply(foo)
_2['confidence_intervals'] = _2['confidence'].apply(foo)
_1 = _1.groupby(['confidence_intervals']).size().reset_index(name='tf-idf_matches').sort_values(['confidence_intervals'],ascending=False)
_2 = _2.groupby(['confidence_intervals']).size().reset_index(name='unique-code_matches').sort_values(['confidence_intervals'],ascending=False)
_1 = _1.merge(_2,how='outer',left_on=['confidence_intervals'],right_on=['confidence_intervals'])
#_1['tf-idf%'] = 100*_1['tf-idf_matches']/(_1['tf-idf_matches'] + _1['unique-code_matches'] )
_1



/var/folders/rj/rrq458bj1mb9fpw62rmd9yb80000gn/T/ipykernel_52721/916000761.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _1['confidence_intervals'] = _1['confidence'].apply(foo)
/var/folders/rj/rrq458bj1mb9fpw62rmd9yb80000gn/T/ipykernel_52721/916000761.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _2['confidence_intervals'] = _2['confidence'].apply(foo)


,confidence_intervals,tf-idf_matches,unique-code_matches
0,95-99,1080,1734.0
1,90-95,850,NaN
2,85-90,1278,NaN
3,80-85,1818,NaN
4,75-80,2260,NaN
5,75% or less,6506,NaN


In [4]:
#df_matches[:10]
random_df = pd.DataFrame()
ranges = [(95,99), (90,95), (85,90), (80,85), (75,80)]
for r in ranges:
    not_upc = df_matches[(~df_matches['notes'].isin(['UPC', 'SKU', 'GTIN', 'MPN']))\
        & (df_matches['source_UPC'] == '') & (df_matches['dest_UPC'] != '')\
            & (df_matches['source_unit_measure'] != '') \
                #& (df_matches['source_file']=='invoices-unique-products.csv')\
                    & (df_matches['confidence'] <= r[1]) & (df_matches['confidence'] > r[0])]
    not_upc = not_upc.sample(80)
    random_df = pd.concat([random_df,not_upc], ignore_index=True)


In [5]:
def foo(x):
    if x >= 75 and x < 80:
        return '75-80'
    if x >=80 and x < 85:
        return '80-85'
    if x >=85 and x < 90:
        return '85-90'
    if x >=90 and x < 95:
        return '90-95'
    if x >=95 and x < 100:
        return '95-99'
random_df['confidence_bin'] = random_df['confidence'].apply(foo)
random_df.groupby(['confidence_bin']).size().reset_index(name='counts')
random_df = random_df.sample(frac=1)
random_df.to_csv('api/random.csv')


In [45]:
random_df = pd.read_csv('api/random.csv', dtype=str)
random_df.groupby(['confidence_bin']).size().reset_index(name='counts')
og_lookup_names = random_df['source_name'].to_list()
og_lookup_names[:10]



['McCormick Spice Shaker Grind 5 Pound Black Pepper',
 'Cheez-It Original Whole Grain Cracker - 0.75 oz.',
 'Base Chicken Paste No Msg',
 'Belvita Breakfast Biscuits - Golden Oat - 1.76 oz. Pack',
 'Malt-O-Meal Chocolate Hot Wheat Cereal 28 oz.',
 'SMITHFIELD CREAM CHEESE',
 'Juice, Lemon 1gal',
 'McCormick Pure Peppermint Extract 1 Pint',
 'Granulated Onions 1lb',
 'cup paper hot 8 oz white']

In [109]:
lookup_names = random['source_name'].to_list()
lookup_names = [l.lower() for l in lookup_names]
# Remove oz 
lookup_names = [l.replace('oz.', '') for l in lookup_names]
lookup_names[:10]



['mccormick spice shaker grind 5 pound black pepper',
 'cheez-it original whole grain cracker - 0.75 ',
 'base chicken paste no msg',
 'belvita breakfast biscuits - golden oat - 1.76  pack',
 'malt-o-meal chocolate hot wheat cereal 28 ',
 'smithfield cream cheese',
 'juice, lemon 1gal',
 'mccormick pure peppermint extract 1 pint',
 'granulated onions 1lb',
 'cup paper hot 8 oz white']

In [69]:
items_dic = {}
for i in items:
    for k,v in i.items():
        items_dic[k] = v
items_dic

{'McCormick Spice Shaker Grind 5 Pound Black Pepper': [],
 'Cheez-It Original Whole Grain Cracker - 0.75 oz.': [{'ean': '0024100792634',
   'title': 'Cheez-It Crackers Whole Grain Single Serve .75 Ounces Case of 175',
   'description': 'Whole Grain Cheez It Crackers.75 ounce single serve packageCase of 175 packagesVery fresh',
   'upc': '024100792634',
   'brand': 'Cheez-It',
   'model': '2410079263',
   'color': '',
   'size': '',
   'dimension': '8.8 X 5.9 X 2.8 inches',
   'weight': '0.1 Pounds',
   'category': 'Food, Beverages & Tobacco > Food Items > Snack Foods > Crackers',
   'currency': '',
   'lowest_recorded_price': 46.69,
   'highest_recorded_price': 99.5,
   'images': ['https://i5.walmartimages.com/asr/7e5de1dd-0037-4a97-aed3-e9d749e66417_1.dd8d43c93e01432be8b317fc11d0dbc7.jpeg?odnHeight=450&odnWidth=450&odnBg=ffffff'],
   'offers': [{'merchant': 'Wal-Mart.com',
     'domain': 'walmart.com',
     'title': 'Cheez-It Crackers Whole Grain Single Serve .75 Ounces Case of 175',


In [114]:
import requests
import time


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0'}
for n, for_url in zip(og_lookup_names, lookup_names):
    if n not in items_dic or items_dic[n] == 429:
        try:
            url = 'https://api.upcitemdb.com/prod/trial/search?s={}&match_mode=1&type=product'.format(for_url)
            x = requests.get(url, timeout=15)

            data = x.json()
            list_items = []
            for i in data['items']:
                list_items.append(i)
            items_dic[n] = list_items
            print('Found', len(list_items), 'items for', for_url)
        except:
            print('No match found for:', for_url, 'Error code:', x.status_code)
            items_dic[n] = x.status_code
        time.sleep(30)
    
""" import requests
import time


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0'}
i = 0
for n, for_url in zip(og_lookup_names[:100], lookup_names[:100]):
    if i < len(items) & len(items[i][n]) == 0:
        _temp = {n:[]}
        try:
            url = 'https://api.upcitemdb.com/prod/trial/search?s={}&match_mode=1&type=product'.format(for_url)
            #x = requests.get(url, headers=headers, proxies=p, timeout=15)
            x = requests.get(url, headers=headers, timeout=15)

            data = x.json()
            list_items = []
            for i in data['items']:
                list_items.append(i)
            _temp[n] = list_items
            items.append(_temp)
            print('Found', len(list_items), 'items for', for_url)
        except:
            print('No match found for:', for_url, 'Error code:', x.status_code)
            items.append(_temp)
    time.sleep(random.randint(10, 15))
     """

Found 5 items for tillamook cheese loaf medium cheddar
No match found for: ea vinegar apple cider 50 grains Error code: 404
Found 5 items for realemon lemon juice
Found 1 items for luzianne tea sweetener - 32 
Found 3 items for monin blue curacao flavored syrup - 1 liter
No match found for: ea spice pepper black table grind 24 mesh Error code: 404
No match found for: njoy blue aspertame sugar substitute - 1 gram Error code: 404
No match found for: c&h sugar brown light Error code: 404
No match found for: bobs red mill tapioca flour - 25 lb. Error code: 404
No match found for: kelloggs keebler graham cracker crumbs - 25 lb. Error code: 404
Found 1 items for bouncer clear measuring cup - 4 qt.
Found 5 items for syrup caramel puremade
No match found for: crumbs bread pankos style blue label Error code: 404
Found 1 items for monin chipotle pineapple flavored syrup - 1 liter
No match found for: foothill farms spaghetti and pizza sauce mix - 11.25  Error code: 404
Found 3 items for monin sug

KeyboardInterrupt: 

In [111]:
len(items_dic)


193

In [115]:
import json
with open('api/outputfile', 'w') as fout:
    json.dump(items_dic, fout)

In [116]:
import json
f = open('api/out (2)')
items_dic = json.load(f)
len(items_dic)


193

In [83]:
for n, for_url in zip(og_lookup_names[:200], lookup_names[:200]):
    if n not in items_dic or items_dic[n] == 429:
        try:
            url = 'https://api.upcitemdb.com/prod/trial/search?s={}&match_mode=1&type=product'.format(for_url)
            x = requests.get(url, timeout=15)

            data = x.json()
            list_items = []
            for i in data['items']:
                list_items.append(i)
            items_dic[n] = list_items
            print('Found', len(list_items), 'items for', for_url)
        except:
            print('No match found for:', for_url, 'Error code:', x.status_code)
            items_dic[n] = x.status_code
        time.sleep(random.randint(10, 20))
    
with open('out', 'w') as fout:
    json.dump(items_dic, fout)

No match found for: maraschino cherries with stem case of 4 / 1 gallon Error code: 429
No match found for: single hellmanns real ancho chipotle sauce jug, 1 gallon -- 1 each Error code: 429


KeyboardInterrupt: 

In [33]:
suppliers= ['Chefs Store', 'Food Service Direct', 'Warehouse 115', 'Round Eye Supply', 'BiRite', 'Webstaurant']
df_all_merged[df_all_merged['supplier'].isin(suppliers)].groupby(['supplier']).size().reset_index(name='counts')

,supplier,counts
0,Chefs Store,1002
1,Food Service Direct,16420
2,Round Eye Supply,9870
3,Warehouse 115,9880


In [41]:
missing_upc = df_all_merged[(df_all_merged['supplier'] == 'Chefs Store') & (df_all_merged['UPC'] == '')]
missing_upc = df_all_merged[(df_all_merged['UPC'] == '')]

len(missing_upc)
#missing_upc[:10]

26474

In [35]:
df_matches.groupby(['source_file']).size().reset_index(name='counts')


,source_file,counts
0,BiRite-merged.csv,2063
1,ChefsStore-merged.csv,242
2,FoodServicesDirect-merged.csv,4069
3,RoundEye-merged.csv,1619
4,Sysco-merged.csv,3
5,invoices-unique-products.csv,210


In [107]:
missing_upc = df_matches[(df_matches['source_file'] == 'ChefsStore-merged.csv') & (df_matches['source_UPC'] == '')\
    & (df_matches['dest_UPC'] != '')]
#missing_upc